1. Import Libraries

In [1]:
#import libraries
import pandas as pd
import numpy as np
import os

2. Import new dataset

In [2]:
#import new data set customers
df_customers = pd.read_csv(R'/Users/samuelcallender/Documents/ Instacart Basket Analysis/02 Data/Original Data/customers.csv', index_col = False)

In [3]:
df_customers.head()

,user_id,First Name,Surnam,Gender,STATE,Age,date_joined,n_dependants,fam_status,income
0,26711,Deborah,Esquivel,Female,Missouri,48,1/1/2017,3,married,165665
1,33890,Patricia,Hart,Female,New Mexico,36,1/1/2017,0,single,59285
2,65803,Kenneth,Farley,Male,Idaho,35,1/1/2017,2,married,99568
3,125935,Michelle,Hicks,Female,Iowa,40,1/1/2017,0,single,42049
4,130797,Ann,Gilmore,Female,Maryland,26,1/1/2017,1,married,40374


In [4]:
df_customers.shape

(206209, 10)

3. Data Wrangling

Renaming and dropping columns

In [5]:
#drop columns
df_customers = df_customers.drop(columns = ['First Name','Surnam'])

In [6]:
df_customers.head()

,user_id,Gender,STATE,Age,date_joined,n_dependants,fam_status,income
0,26711,Female,Missouri,48,1/1/2017,3,married,165665
1,33890,Female,New Mexico,36,1/1/2017,0,single,59285
2,65803,Male,Idaho,35,1/1/2017,2,married,99568
3,125935,Female,Iowa,40,1/1/2017,0,single,42049
4,130797,Female,Maryland,26,1/1/2017,1,married,40374


Check for missing data and values

In [7]:
#missing data
df_customers.isnull().sum()

user_id         0
Gender          0
STATE           0
Age             0
date_joined     0
n_dependants    0
fam_status      0
income          0
dtype: int64

No missing values in this data set

In [8]:
#rename the columns to be more consistent
df_customers.rename(columns = {'Gender' : 'gender', 'STATE' : 'state','Age' : 'age',
                               'n_dependants' : 'number_dependants', 'fam_status' : 'family_status'}, inplace = True)

In [9]:
df_customers.head()

,user_id,gender,state,age,date_joined,number_dependants,family_status,income
0,26711,Female,Missouri,48,1/1/2017,3,married,165665
1,33890,Female,New Mexico,36,1/1/2017,0,single,59285
2,65803,Male,Idaho,35,1/1/2017,2,married,99568
3,125935,Female,Iowa,40,1/1/2017,0,single,42049
4,130797,Female,Maryland,26,1/1/2017,1,married,40374


In [10]:
#change user_id data type
df_customers['user_id'] = df_customers['user_id'].astype('str')
df_customers['user_id'].dtype

dtype('O')

4. Data Consistency

In [11]:
df_customers.describe()

,age,number_dependants,income
count,206209.000000,206209.000000,206209.000000
mean,49.501646,1.499823,94632.852548
std,18.480962,1.118433,42473.786988
min,18.000000,0.000000,25903.000000
25%,33.000000,0.000000,59874.000000
50%,49.000000,1.000000,93547.000000
75%,66.000000,3.000000,124244.000000
max,81.000000,3.000000,593901.000000


In [12]:
#search for duplicates
df_customers[df_customers.duplicated()]

,user_id,gender,state,age,date_joined,number_dependants,family_status,income


No duplicates found

In [13]:
df_customers.shape

(206209, 8)

5. Looking for mixed data types

In [14]:
for col in df_customers.columns.tolist():
  weird = (df_customers[[col]].applymap(type) != df_customers[[col]].iloc[0].apply(type)).any(axis = 1)
  if len (df_customers[weird]) > 0:
    print (col)

No mixed datatypes found.

6. Merge datasets

In [20]:
#import orders_products_merged.pkl 
df_ords_merged = pd.read_pickle(r'/Users/samuelcallender/Documents/ Instacart Basket Analysis/02 Data/Prepared Data/orders_products_merged_updated_3.pkl')

In [21]:
#merge datasets
df_ords_prods_custm = pd.merge(df_ords_merged, df_customers, on = 'user_id', how = 'outer')

In [22]:
df_ords_prods_custm.head()

,Unnamed: 0.1,Unnamed: 0,order_id,user_id,eval_set,order_number,order_day_of_week,order_hour_of_day,days_since_prior_order,product_id,...,spending_flag,median_prior_orders,order_frequency_flag,gender,state,age,date_joined,number_dependants,family_status,income
0,0,0,2539329,1,prior,1,2,8,NaN,196,...,Low spender,20.5,Non-frequent customer,Female,Alabama,31,2/17/2019,3,married,40423
1,1,1,2398795,1,prior,2,3,7,15.0,196,...,Low spender,20.5,Non-frequent customer,Female,Alabama,31,2/17/2019,3,married,40423
2,2,2,473747,1,prior,3,3,12,21.0,196,...,Low spender,20.5,Non-frequent customer,Female,Alabama,31,2/17/2019,3,married,40423
3,3,3,2254736,1,prior,4,4,7,29.0,196,...,Low spender,20.5,Non-frequent customer,Female,Alabama,31,2/17/2019,3,married,40423
4,4,4,431534,1,prior,5,4,15,28.0,196,...,Low spender,20.5,Non-frequent customer,Female,Alabama,31,2/17/2019,3,married,40423


In [23]:
#check the values counts
df_ords_prods_custm['_merge'].value_counts()

both          32406041
left_only            0
right_only           0
Name: _merge, dtype: int64

In [24]:
#check the shape
df_ords_prods_custm.shape

(32406041, 34)

In [25]:
#export data as pkl
df_ords_prods_custm.to_pickle(r'/Users/samuelcallender/Documents/ Instacart Basket Analysis/02 Data/Prepared Data/orders_products_customers.pkl')